<a href="https://colab.research.google.com/github/rezabonyadi/language-models-experiments/blob/main/llm_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.1 MB/s eta 0:00:00


# Load data

In [ ]:
import os
import numpy as np
import pandas as pd
import requests
from sklearn.datasets import fetch_20newsgroups
from datasets import load_dataset
import tiktoken # TikTone is MUCH faster and MUCH more memory efficient than tokenizers from HuggingFace
# Encode with tiktoken gpt2 bpe
# enc = tiktoken.get_encoding('gpt2')
from datasets import load_dataset

def get_data(name):
    if name=='spotify':
        df = pd.read_csv('/content/spotify_millsongdata.csv')
        data = df['text'].str.cat(sep='\n')
    if name=='ted':
        df = pd.read_csv('/content/transcripts.csv')
        data = df['transcript'].str.cat(sep='\n')
    if name=='sheaks':
        input_file_path = '/content/data_input.txt'
        data_url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
        with open(input_file_path, 'w') as f:
            f.write(requests.get(data_url).text)
        with open(input_file_path, 'r') as f:
            data = f.read()
    if name=='news_group':
        # Fetch the training data
        newsgroups_train = fetch_20newsgroups(subset='train')
        data = newsgroups_train.data
    if name=='CNN':
      dataset = load_dataset("cnn_dailymail", "3.0.0")
      data = '\n\n'.join(dataset['train']['article'][:1000])
    if name=='lamini':
        dataset = load_dataset("MBZUAI/LaMini-instruction")
        df = pd.DataFrame({'instruction': dataset['train']['instruction'], 'response': dataset['train']['response']})
        # TODO: Fix this
    if name=="wiki":
      dataset = load_dataset("wikitext", "wikitext-103-v1")
      data = '\n\n'.join(dataset['train']['text'][:])
    if name=='tiny':
        dataset = load_dataset("roneneldan/TinyStories")
        data = '\n\n'.join(dataset['train']['text'])
    if name == 'cbt':
      dataset = load_dataset('cbt', 'raw')
      data = '\n\n'.join(dataset['train']['content'][:10])

    return data

# data = get_data('sheaks')
data = get_data('tiny')

# data = get_data('cbt')
# data = get_data('wiki')
# data = get_data('CNN')

n = len(data)

data = data[:int(n*0.5)]
n = len(data)

train_data = data[:int(n*0.9)]
val_data = data[int(n*0.9):]
tokenizer = tiktoken.get_encoding("gpt2")  # Use GPT-2 tokenizer or custom tokenizer

# enc = TokenizerEncoder(vocab_file='gpt2-vocab.json', merges_file='gpt2-merges.txt')
train_tokens_full = tokenizer.encode(train_data)
val_tokens_full = tokenizer.encode(val_data)
print(f"train has {len(train_tokens_full):,} tokens")
print(f"val has {len(val_tokens_full):,} tokens")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

(…)-00000-of-00004-2d5a1467fff1081b.parquet:   0%|          | 0.00/249M [00:00<?, ?B/s]

(…)-00001-of-00004-5852b56a2bd28fd9.parquet:   0%|          | 0.00/248M [00:00<?, ?B/s]

(…)-00002-of-00004-a26307300439e943.parquet:   0%|          | 0.00/246M [00:00<?, ?B/s]

(…)-00003-of-00004-d243063613e5a057.parquet:   0%|          | 0.00/248M [00:00<?, ?B/s]

(…)-00000-of-00001-869c898b519ad725.parquet:   0%|          | 0.00/9.99M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2119719 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/21990 [00:00<?, ? examples/s]

train has 214,353,431 tokens
val has 23,824,350 tokens


In [ ]:
train_ids = train_tokens_full
val_ids = val_tokens_full

# Reduce vocab test

You can consider reducing your vocab size to the actual tokens that do appear in the training data. You can even consider reducing that to most frequent tokens. For example, if you consider tokens that appear over 1000 times in the text with 214M tokens, you will reduce the vocab size from 25000 to 4500! This reduces the size of the embedding drastically, leading to training your model with much smaller memory footprint.

It is the freq_threshold parameter below. Set to 0 for using all tokens from your training data.

In [ ]:
import os
import numpy as np
from collections import Counter
import torch
from torch.utils.data import Dataset
from transformers import LlamaForCausalLM, LlamaConfig, TrainingArguments, Trainer
from transformers import get_linear_schedule_with_warmup

# ----------------------------
# 1. Reduced Vocabulary Setup
# ----------------------------

# Assume train_tokens_full and val_tokens_full are your lists of GPT-2 token IDs.
freq_threshold = 1  # tokens with frequency < threshold will be <unk>
token_freq = Counter(train_tokens_full)

frequent_tokens = []
non_frequent_tokens = []
for token_id, freq in token_freq.items():
    if freq >= freq_threshold:
        frequent_tokens.append(token_id)
    else:
        non_frequent_tokens.append(token_id)

# Stable new ID assignment
frequent_tokens = sorted(frequent_tokens)

# Build new vocabulary mapping: frequent tokens + <unk>
old2new = {old_id: i for i, old_id in enumerate(frequent_tokens)}
unk_id = len(frequent_tokens)  # new ID for <unk>
vocab_size_new = unk_id + 1

print(f"Number of frequent tokens: {len(frequent_tokens)}")
print(f"New vocab size: {vocab_size_new} (including <unk>)")

print(f"Some non-frequent tokens for frequency threshold of {freq_threshold}: ", tokenizer.decode(non_frequent_tokens[-5:]))

def remap_to_reduced_vocab(token_ids, old2new_map, unk_token_id):
    """Map original token IDs to new IDs, using <unk> for infrequent tokens."""
    return [old2new_map.get(t, unk_token_id) for t in token_ids]

train_ids = remap_to_reduced_vocab(train_tokens_full, old2new, unk_id)
val_ids   = remap_to_reduced_vocab(val_tokens_full, old2new, unk_id)

print(f"Remapped train tokens: {len(train_ids)}")
print(f"Remapped val tokens:   {len(val_ids)}")

# Build the inverse mapping for decoding later (if needed)
new2old = {new_id: old_id for old_id, new_id in old2new.items()}
new2old[unk_id] = None  # interpret None as unknown

# ----------------------------
# 2. Define a PyTorch Dataset for Trainer
# ----------------------------

class TokenDataset(Dataset):
    def __init__(self, token_ids, seq_length):
        """
        token_ids: List of integer token IDs.
        seq_length: The fixed sequence length for training examples.
        """
        self.token_ids = token_ids
        self.seq_length = seq_length
        self.num_chunks = len(token_ids) // seq_length

    def __len__(self):
        return self.num_chunks

    def __getitem__(self, idx):
        start = idx * self.seq_length
        end = start + self.seq_length
        chunk = self.token_ids[start:end]
        # For causal LM, labels are identical to input_ids
        return {"input_ids": torch.tensor(chunk, dtype=torch.long),
                "labels": torch.tensor(chunk, dtype=torch.long)}

SEQ_LENGTH = 256
train_dataset = TokenDataset(train_ids, SEQ_LENGTH)
val_dataset   = TokenDataset(val_ids, SEQ_LENGTH)


Number of frequent tokens: 26474
New vocab size: 26475 (including <unk>)
Some non-frequent tokens for frequency threshold of 1:  
Remapped train tokens: 214353431
Remapped val tokens:   23824350


In [ ]:
def generate_text(model, prompt_ids, max_new_tokens=50):
    """
    Autoregressively generate tokens (in reduced vocabulary space) from a prompt.
    Returns a list of token IDs including the prompt.
    """
    model.eval()
    device = next(model.parameters()).device
    input_ids = torch.tensor(prompt_ids, dtype=torch.long, device=device).unsqueeze(0)

    with torch.no_grad():
        for _ in range(max_new_tokens):
            outputs = model(input_ids)
            logits = outputs.logits  # shape: [1, seq_length, vocab_size_new]
            next_token_logits = logits[:, -1, :]
            next_token = torch.argmax(next_token_logits, dim=-1, keepdim=True)
            input_ids = torch.cat([input_ids, next_token], dim=1)

    return input_ids.squeeze(0).tolist()

def decode_reduced_tokens(reduced_token_ids, new2old_map, gpt2_tokenizer, unk_token_id):
    """
    Convert reduced token IDs back to original GPT-2 token IDs, then decode.
    For tokens that map to <unk>, a placeholder token (e.g., 198) is used.
    """
    old_ids = []
    for token in reduced_token_ids:
        old_id = new2old_map.get(token, None)
        if old_id is None:
            old_ids.append(198)
        else:
            old_ids.append(old_id)
    return gpt2_tokenizer.decode(old_ids)


In [ ]:
# ----------------------------
# 3. Set up Llama Model with Modified Config
# ----------------------------

# Create a Llama config with the new vocabulary size.
config = LlamaConfig(
    vocab_size=vocab_size_new,
    hidden_size=512,       # Use a smaller hidden size for demo purposes; adjust as needed.
    num_hidden_layers=6,
    num_attention_heads=8,
    intermediate_size=2048,
    max_position_embeddings=SEQ_LENGTH
)

model = LlamaForCausalLM(config)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(26475, 512)
    (layers): ModuleList(
      (0-5): 6 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=512, out_features=512, bias=False)
          (k_proj): Linear(in_features=512, out_features=512, bias=False)
          (v_proj): Linear(in_features=512, out_features=512, bias=False)
          (o_proj): Linear(in_features=512, out_features=512, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=512, out_features=2048, bias=False)
          (up_proj): Linear(in_features=512, out_features=2048, bias=False)
          (down_proj): Linear(in_features=2048, out_features=512, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((512,), eps=1e-06)
        (post_attention_layernorm): LlamaRMSNorm((512,), eps=1e-06)
      )
    )
    (norm): LlamaRMSNorm((512,), eps=1e-06)
    (rotary_emb): LlamaRotaryEmbe

In [ ]:
from transformers import TrainerCallback
import wandb

class SampleGenerationCallback(TrainerCallback):
    def __init__(self, sample_prompt, logging_interval=100):
        """
        sample_prompt: A list of reduced vocabulary token IDs to use as the prompt.
        logging_interval: Log sample outputs every N steps.
        """
        self.sample_prompt = sample_prompt
        self.logging_interval = logging_interval

    def on_log(self, args, state, control, **kwargs):
        # Log only every logging_interval steps
        if state.global_step % self.logging_interval != 0:
            return

        # Retrieve the model from kwargs
        model = kwargs.get("model", None)
        if model is None:
            return

        # Generate new tokens using the provided generate_text function.
        # Make sure generate_text is in scope.
        generated_ids = generate_text(model, self.sample_prompt, max_new_tokens=50)

        # Decode generated tokens using the provided decode_reduced_tokens function.
        # Make sure decode_reduced_tokens, new2old, tokenizer, and unk_id are in scope.
        sample_text = decode_reduced_tokens(generated_ids, new2old, tokenizer, unk_id)

        # Log the sample output to wandb and print it.
        wandb.log({"sample_output": sample_text, "step": state.global_step})
        print(f"Sample output at step {state.global_step}:\n{sample_text}\n")

import numpy as np
start_idx = np.random.randint(0, len(val_ids) - 20)
sample_prompt = val_ids[start_idx : start_idx + 20]

sample_callback = SampleGenerationCallback(sample_prompt=sample_prompt, logging_interval=10)

prompt_text = decode_reduced_tokens(
    sample_prompt,
    new2old_map=new2old,
    gpt2_tokenizer=tokenizer,
    unk_token_id=unk_id
)
prompt_text

' to have fun!\n\nOnce upon a time there was a little man. He liked to build'

In [ ]:
# ----------------------------
# 4. Trainer and TrainingArguments
# ----------------------------

training_args = TrainingArguments(
    output_dir=f"./llama_reduced_vocab_ft_{freq_threshold}",
    num_train_epochs=3,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_strategy="steps",  # log every N steps
    logging_steps=10,          # log every 10 steps (adjust as desired)
    logging_first_step=True,   # log the first training step
    report_to="wandb"    # or "none" if you don't want to use external reporting tools
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    callbacks=[sample_callback]
)

# Start training
trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: vardiar to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,1.741000,1.765214
2,1.600100,1.630495
3,1.535500,1.588001


Streaming output truncated to the last 5000 lines.
The tower was very tall and it was almost as tall as the little man. He was so proud of it. He wanted

Sample output at step 32230:
 to have fun!

Once upon a time there was a little man. He liked to build things. One day he decided to build a big tower. He worked very hard and was very proud of it.

The man worked very hard and he was very proud of his tower. He wanted to show it to everyone. He was very

Sample output at step 32240:
 to have fun!

Once upon a time there was a little man. He liked to build things. One day he decided to build a big tower. He worked very hard and was very proud of it.

The man worked very hard and he was very proud of his tower. He wanted to show it to everyone. He was so

Sample output at step 32250:
 to have fun!

Once upon a time there was a little man. He liked to build things. One day he decided to build a big tower. He worked very hard and was very proud of it.

But then something bad happened. Th

TrainOutput(global_step=39252, training_loss=1.8063682765755633, metrics={'train_runtime': 28837.3911, 'train_samples_per_second': 87.108, 'train_steps_per_second': 1.361, 'total_flos': 1.4942538345480192e+17, 'train_loss': 1.8063682765755633, 'epoch': 3.0})

In [ ]:

# ----------------------------
# 5. Generation Example
# ----------------------------

# Assume you have a GPT-2 tokenizer loaded as `tokenizer`
# For example: from transformers import GPT2Tokenizer; tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
start_idx = np.random.randint(0, len(val_ids) - 20)
prompt_ids = val_ids[start_idx : start_idx + 20]

generated_reduced = generate_text(model, prompt_ids, max_new_tokens=50)


generated_text = decode_reduced_tokens(
    generated_reduced,
    new2old_map=new2old,
    gpt2_tokenizer=tokenizer,
    unk_token_id=unk_id
)

print("PROMPT + GENERATED TEXT:\n", generated_text)


PROMPT + GENERATED TEXT:
  him pick some flowers for his mom. The boy said, "Thank you!" and ran home with his mom.

When they got home, the boy's mom asked him to help her with the laundry. The boy said, "I'm sorry, I can't help you with the laundry." His mom said, "It's okay,


In [ ]:
prompt_text = decode_reduced_tokens(
    prompt_ids,
    new2old_map=new2old,
    gpt2_tokenizer=tokenizer,
    unk_token_id=unk_id
)
prompt_text

' him pick some flowers for his mom. The boy said, "Thank you!" and ran home with'